In [1]:
from typing import List, Optional

from sly_llama import llm_call
from langchain.llms import OpenAI

from pydantic import BaseModel

llm = OpenAI()

#### Lets define what the add function does and wrap it in an llm call

In [2]:
@llm_call(llm)
def add(x: str, y: str) -> str:
    """
    calculate {x} + {y}
    only return the number and nothing else
    """

In [3]:
add(1, 2)

'\n3'

In [4]:
add(1, 3) + add(1,1)

'\n4\n2'

#### Problem: strings don't add, lets try again but with ints


In [5]:
@llm_call(llm)
def add(x: str, y: str) -> int:
    """
    calculate {x} + {y}
    only return the number and nothing else
    """

add(1, 3) + add(1,1)

6

Lets make a recipe

In [6]:

@llm_call(llm)
def get_recipe(dish: str, units: str) -> str:
    """
    Write a resipe for this {dish}
    Be sure to include all the ingridients in {units} units.


    ingridients: < neccesary ingridients>
    intructions: < the instructions for making the dish>
    vegan : <this value must be one of [True, False] indicating weather the recipe is vegan>

    """

In [7]:
print(get_recipe("jank", "metric"))

'\nIngridients:\n\n- 2 cups of all-purpose flour\n- 2 teaspoons baking powder\n- 1/4 teaspoon baking soda\n- 1/2 teaspoon salt\n- 1/4 cup vegan butter\n- 1/2 cup almond milk\n- 2 tablespoons sugar\n- 2 tablespoons vegan yogurt\n\nInstructions:\n\n1. Preheat oven to 350°F (175°C).\n\n2. In a large bowl, whisk together the all-purpose flour, baking powder, baking soda, and salt.\n\n3. Add the vegan butter and mix with a pastry blender or a fork until it resembles a course meal.\n\n4. Add the almond milk, sugar, and vegan yogurt and mix until a soft dough forms.\n\n5. Turn the dough out onto a lightly floured surface and knead until it forms a ball.\n\n6. Roll the dough out to 1/2 inch thickness and cut into rounds using a biscuit cutter or glass.\n\n7. Place on a lightly greased baking sheet and bake for 12-15 minutes or until lightly browned.\n\nVegan: True'

#### That's great but what if we want to parse the output to a pydantic class

#### Let define the output class and how we want to parse the llm output

In [28]:
from pydantic import BaseModel

class Recipe(BaseModel):
    ingridients: str | List[str]
    instructions : str | List[str]
    vegan: bool

    @classmethod
    def from_llm_output(cls, llm_output: str):
        recipe = {}
        parts = llm_output.casefold().partition('instructions')
        recipe['ingridients']  = parts[0].replace('ingridients', '').replace('[],"', '').strip().split('\n')
        recipe['instructions'] = parts[2].partition('vegan')[0].replace('[],"', '').strip().split('\n')
        recipe['vegan']  = bool(parts[2].partition('vegan')[1].replace('[],"\n', '').strip())
        return cls.parse_obj(recipe)



#### And ammend the return type

In [29]:
@llm_call(llm)
def get_recipe(dish: str, units: str) -> Recipe:
     """
     Write a resipe for this {dish}
     Be sure to include all the ingridients in {units} units.

     You should provide your response in JSON Format

     ingridients: < neccesary ingridients>
     intructions: < the instructions for making the dish>
     vegan : <this value must be one of [True, False] indicating weather the recipe is vegan>
     """

In [31]:
recipe = get_recipe('kchapuri', 'metric')
recipe.instructions

['": [',
 '        "mix the flour, salt, and oil in a bowl.",',
 '        "slowly add the warm water to the bowl and knead the dough for 10 minutes.",',
 '        "cover the dough and let it rest for 20 minutes.",',
 '        "divide the dough into 4 parts, shape into balls and roll out each ball into a thin round shape.",',
 '        "heat a large non-stick pan over medium heat.",',
 '        "put one round of dough on the pan and cook for 1-2 minutes.",',
 '        "flip over and cook the other side for 1-2 minutes.",',
 '        "remove from the pan and repeat with the remaining dough."',
 '    ],',
 '    "']

#### Hmm that was a lot of work and looks like we did not do a good job, lets ask it to give us some juicy JSON

In [10]:
from sly_llama import JsonBaseModel

class Recipe(JsonBaseModel):
    ingridients: str | List[str]
    instructions : str | List[str]
    vegan: bool

#### Llamas are not so good at json so may be let it learn from its mistakes

In [11]:
@llm_call(llm)
def get_recipe(dish: str, units: str, error_message: str) -> Recipe:
    """
    Write a resipe for this {dish}
    Be sure to include all the ingridients in {units} units.

    You should provide your response in JSON Format

    ingridients: < neccesary ingridients>
    intructions: < the instructions for making the dish>
    vegan : <this value must be one of [True, False] indicating weather the recipe is vegan>

    {error_message}
    """

In [9]:
from sly_llama import LlmException

recipe = None
error_message = ''

while not recipe:
    try:
        recipe = get_recipe('kchapuri', 'metric', error_message)
        
    except LlmException as e:
        error_message = e.message
        print(error_message)
recipe

Recipe(ingridients=['300g Flour', '1/4 teaspoon of salt', '150ml warm water', '2 tablespoons of vegetable oil', '1/2 teaspoon of sugar'], instructions=['In a bowl, mix together the flour, salt, sugar and oil.', 'Slowly add the warm water and knead for 10 minutes or until the dough is soft and elastic.', 'Roll the dough into a ball and let it rest for 15 minutes in an oiled bowl.', 'Divide the dough into two parts and roll each out to a round shape.', 'Brush each chapati with oil and cook in a heavy skillet over medium heat for about 1 minute on each side.', 'Serve warm.'], vegan=True)